In [ ]:
!pip install pyodbc

In [ ]:
import pyodbc
import requests

# Database connection

In [ ]:
DATABASE_CONFIG = {
    'server': 'your_server_name',
    'database': 'your_database_name',
    'username': 'your_username',
    'password': 'your_password',
    'driver': '{ODBC Driver 18 for SQL Server}',
    'timeout': 60
}

# Singleton pattern for database connection

In [ ]:
class DatabaseConnection:
    """
    Singleton class for establishing and managing the database connection.
    Ensures a single instance of the connection exists across the application.
    """
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            try:
                connection_string = (
                    f"DRIVER={DATABASE_CONFIG['driver']};"
                    f"SERVER={DATABASE_CONFIG['server']};"
                    f"DATABASE={DATABASE_CONFIG['database']};"
                    f"UID={DATABASE_CONFIG['username']};"
                    f"PWD={DATABASE_CONFIG['password']};"
                    f"Encrypt=yes;"
                    f"TrustServerCertificate=no;"
                    f"Connection Timeout={DATABASE_CONFIG['timeout']};"
                )
                cls._instance = pyodbc.connect(connection_string)
                print("Database connection successful.")
            except Exception as e:
                print(f"Failed to connect to database: {e}")
        return cls._instance

    @staticmethod
    def close_connection():
        """Close the database connection."""
        if DatabaseConnection._instance:
            DatabaseConnection._instance.close()
            print("Database connection closed.")

# Google Books API client

In [ ]:
def fetch_books_from_google_api(query="Python programming", max_results=5):
    """
    Fetch book data from Google Books API.
    - query: Search term for fetching books.
    - max_results: Limit the number of results.
    """
    api_url = f"https://www.googleapis.com/books/v1/volumes?q={query}&maxResults={max_results}"
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        books = [
            {
                'title': item['volumeInfo'].get('title', 'N/A'),
                'authors': ', '.join(item['volumeInfo'].get('authors', ['Unknown Author'])),
                'genre': ', '.join(item['volumeInfo'].get('categories', ['N/A'])),
                'year': item['volumeInfo'].get('publishedDate', 'N/A')[:4],
                'rating': None  # Assume no rating available
            }
            for item in response.json().get('items', [])
        ]
        print(f"Fetched {len(books)} books from Google Books API.")
        return books
    except requests.RequestException as e:
        print(f"Error fetching data from Google Books API: {e}")
        return []

# CRUD operations for Books table

In [ ]:
def setup_books_table(connection):
    """Creates the Books table in the database if it does not already exist."""
    cursor = connection.cursor()
    create_table_sql = """
    IF OBJECT_ID('dbo.Books', 'U') IS NULL
    BEGIN
        CREATE TABLE dbo.Books (
            BookID INT IDENTITY(1,1) PRIMARY KEY,
            Title NVARCHAR(255) NOT NULL,
            Author NVARCHAR(255) NOT NULL,
            Genre NVARCHAR(100),
            PublicationYear INT,
            Rating FLOAT
        );
    END;
    """
    cursor.execute(create_table_sql)
    connection.commit()
    print("Books table created (or already exists).")
    cursor.close()

def insert_books(connection, books):
    """Inserts a list of books into the Books table."""
    cursor = connection.cursor()
    insert_sql = "INSERT INTO Books (Title, Author, Genre, PublicationYear, Rating) VALUES (?, ?, ?, ?, ?);"
    for book in books:
        cursor.execute(insert_sql, (book['title'], book['authors'], book['genre'], book['year'], book['rating']))
    connection.commit()
    print(f"Inserted {len(books)} books into the database.")
    cursor.close()

def retrieve_books(connection):
  """Retrieves and displays all books from the Books table."""
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM Books;")
    rows = cursor.fetchall()
    print("Books in the database:")
    for row in rows:
        print(f"ID: {row.BookID}, Title: {row.Title}, Author: {row.Author}, Genre: {row.Genre}, Year: {row.PublicationYear}, Rating: {row.Rating}")
    cursor.close()

def delete_books(connection, author="Unknown Author"):
  """Deletes books from the Books table based on the author."""
    cursor = connection.cursor()
    cursor.execute("DELETE FROM Books WHERE Author = ?;", (author,))
    connection.commit()
    print(f"Deleted books by author '{author}'.")
    cursor.close()

In [ ]:
# Testing functionality

In [ ]:
import logging
import time

## Configure logging

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def test_igottabook_app():
  """Tests core functionalities of the IGottaBook application."""
  logging.info("Starting IGottaBook tests...")


    # Connect to the database
    connection = DatabaseConnection()
    if not connection:
        logging.error("Failed to connect to database. Tests cannot proceed.")
        return

    try:
        # Test 1: Table Setup
        logging.info("Testing table setup...")
        setup_books_table(connection)

        # Test 2: API Fetch with Valid Query
        logging.info("Testing API fetch with valid query...")
        start_time = time.time()
        books = fetch_books_from_google_api("Python programming")
        fetch_time = time.time() - start_time
        logging.info(f"Fetched {len(books)} books in {fetch_time:.2f} seconds.")
        assert books, "No books fetched from the API."

        # Test 3: API Fetch with Empty Query
        logging.info("Testing API fetch with empty query...")
        empty_books = fetch_books_from_google_api("")
        assert not empty_books, "API returned books for an empty query."

        # Test 4: Insert Books
        logging.info("Testing book insertion...")
        start_time = time.time()
        insert_books(connection, books)
        insert_time = time.time() - start_time
        logging.info(f"Inserted {len(books)} books into the database in {insert_time:.2f} seconds.")

        # Test 5: Retrieve Books
        logging.info("Testing data retrieval...")
        retrieved_books = retrieve_books(connection)
        assert len(retrieved_books) == len(books), "Mismatch in inserted and retrieved books."

        # Test 6: Cleanup
        logging.info("Testing data cleanup...")
        delete_books(connection)
        cleanup_books = retrieve_books(connection)
        assert not cleanup_books, "Cleanup failed; some books remain in the database."

    except Exception as e:
        logging.error(f"An error occurred during testing: {e}")
    finally:
        DatabaseConnection.close_connection()
        logging.info("Testing completed. Database connection closed.")


# Instructions for execution
To run this code:
1. Ensure Python and required libraries (pyodbc, requests) are installed.
2. Configure database access credentials in DATABASE_CONFIG.
3. Run the script using `python igottabook.py`.

# Main execution

In [ ]:
if __name__ == "__main__":
    test_igottabook_app()